In [1]:
import pandas as pd
import re

# === Путь к твоим входным данным ===
# Файл с shell-командами
script_path = "reconstruct_belle1_mc_hadron_y5s.sh"

# CSV или DataFrame с нужными экспериментами и ранами
# Заменить на путь к реальному файлу, если требуется
df = pd.read_csv("exp_run_list.csv")  # должен содержать столбцы: 'experiment' и 'run'

# === Считываем строки скрипта ===
with open(script_path) as f:
    lines = f.readlines()


In [2]:

# === Парсим команды и извлекаем exp / run_start / run_end ===
commands = []
for line in lines:
    match = re.match(r'\S+\s+(\d+)\s+(\d+)\s+(\d+)', line)
    if match:
        exp = int(match.group(1))
        run_start = int(match.group(2))
        run_end = int(match.group(3))
        commands.append({
            "line": line.strip(),
            "exp": exp,
            "run_start": run_start,
            "run_end": run_end
        })
    else:
        print(f"⚠️ Строка пропущена (не распознана): {line.strip()}")

# === Сопоставляем с DataFrame ===
selected_lines = []

for cmd in commands:
    subset = df[df["__experiment__"] == cmd["exp"]]
    if any((cmd["run_start"] <= r <= cmd["run_end"]) for r in subset["__run__"]):
        selected_lines.append(cmd["line"])

# === Записываем результат ===
output_path = "filtered_script.sh"

with open(output_path, "w") as f:
    for line in selected_lines:
        f.write(line + "\n")

print(f"✅ Отобрано {len(selected_lines)} команд из {len(commands)}. Результат сохранён в: {output_path}")


⚠️ Строка пропущена (не распознана): #!/bin/bash
⚠️ Строка пропущена (не распознана): 
⚠️ Строка пропущена (не распознана): TY=$1
⚠️ Строка пропущена (не распознана): STREAM=$2
⚠️ Строка пропущена (не распознана): TIME=$3
⚠️ Строка пропущена (не распознана): 
⚠️ Строка пропущена (не распознана): #
⚠️ Строка пропущена (не распознана): 
✅ Отобрано 87 команд из 1060. Результат сохранён в: filtered_script.sh
